In [17]:
from ugot import ugot
import random
import time

got = ugot.UGOT()

got.initialize('192.168.1.120')


192.168.1.120:50051


In [ ]:
class UGOTCommandCenter:
    def __init__(self, got):
        self.got = got
        self.running = False

        # List of simple commands (for help)
        self.command_list = ['forward', 'backward', 'left', 'right']

        # Map command names to handler methods
        self._handlers = {
            'exit':     self._handle_exit,
            'help':     self._handle_help,
            'forward':  self._handle_forward,
            'backward': self._handle_backward,
            'left':     self._handle_left,
            'right':    self._handle_right,
        }

    def _handle_exit(self):
        print("Exiting robot...")
        self.running = False

    def _handle_help(self):
        print("Here are the available commands:")
        for name in self.command_list:
            print(f"- {name}")

    def _handle_forward(self, times: int):
        got.play_audio_tts(f"Moving forward {times}", voice_type=0, wait=True)
        self.got.mecanum_move_speed_times(0, 20, times, 1)

    def _handle_backward(self, times: int):
        got.play_audio_tts(f"Moving backward {times}", voice_type=0, wait=True)
        self.got.mecanum_move_speed_times(1, 20, times, 1)

    def _handle_left(self, times: int):
        got.play_audio_tts(f"Moving left {times}", voice_type=0, wait=True)
        self.got.mecanum_turn_speed_times(2, 30, times, 2)

    def _handle_right(self, times: int):
        got.play_audio_tts(f"Moving right {times}", voice_type=0, wait=True)
        self.got.mecanum_turn_speed_times(3, 30, times, 2)

    def run(self):
        """Start the command loop."""
        self.running = True
        print("Welcome to the UGOT Command Center! Type 'exit' to quit. Type 'help' for a list of commands.")
        while self.running:
            line = input("What are your orders?\n> ").strip().lower()
            if not line:
                continue

            tokens = line.split()
            i = 0
            # Process tokens one-by-one, allowing multiple commands per line
            while i < len(tokens):
                cmd = tokens[i]
                handler = self._handlers.get(cmd)
                if handler is None:
                    print(f"Unknown command: '{cmd}'. Type 'help' to see available commands.")
                    i += 1
                else:
                    # 'exit' and 'help' take no numeric argument
                    if cmd in ('exit', 'help'):
                        handler()
                        i += 1
                    else:
                        # movement/turn commands expect a numeric argument next
                        if i + 1 < len(tokens) and tokens[i+1].isdigit():
                            arg = int(tokens[i+1])
                            handler(arg)
                            i += 2
                        else:
                            print(f"Command '{cmd}' requires a numeric argument.")
                            i += 1

center = UGOTCommandCenter(got)
center.run()

Welcome to the UGOT Command Center! Type 'exit' to quit. Type 'help' for a list of commands.
Exiting robot...


In [ ]:
def ugot_voice_activated():
    command = got.start_audio_asr()
    print(command)

    parts = command.lower().split()
    if not parts: 
        return

    # “exit” to quit
    if "exit" in parts:
        got.play_audio_tts("Shutting down. Goodbye!", voice_type=0, wait=True)
        return False

    # look for numeric argument
    amount = next((int(p) for p in parts if p.isdigit()), None)
    print(amount)

    if "forward" in parts and amount is not None:
        got.play_audio_tts(f"Moving forward {amount}", voice_type=0, wait=True)
        got.mecanum_move_speed_times(0, 20, amount, 1)
    elif "backward" in parts and amount is not None:
        got.play_audio_tts(f"Moving backward {amount}", voice_type=0, wait=True)
        got.mecanum_move_speed_times(1, 20, amount, 1)
    elif "left" in parts and amount is not None:
        got.play_audio_tts(f"Turning left {amount} times", voice_type=0, wait=True)
        got.mecanum_turn_speed_times(2, 30, amount, 2)
    elif "right" in parts and amount is not None:
        got.play_audio_tts(f"Turning right {amount} times", voice_type=0, wait=True)
        got.mecanum_turn_speed_times(3, 30, amount, 2)
    else:
        got.play_audio_tts("Sorry, I didn't get that. Try commands like 'forward five' or 'left three'.")

ugot_voice_activated()

Forward 10
10


True

In [ ]:
class VoiceController:
    def __init__(self, got_interface, voice_type=0):
        self.got = got_interface
        self.voice_type = voice_type
        self.running = True

    def listen_and_process(self):
        command = self.got.start_audio_asr()
        print(f"Raw command: {command!r}")

        parts = command.lower().split()
        if not parts:
            return True 

        if "exit" in parts:
            self._handle_exit()
            return False

        amount = self._extract_amount(parts)
        print(f"Parsed amount: {amount}")

        if "forward" in parts and amount is not None:
            self._move_forward(amount)
        elif "backward" in parts and amount is not None:
            self._move_backward(amount)
        elif "left" in parts and amount is not None:
            self._turn_left(amount)
        elif "right" in parts and amount is not None:
            self._turn_right(amount)
        else:
            self._unknown_command()

        return True

    def run(self):
        while self.running:
            self.running = self.listen_and_process()

    def _extract_amount(self, parts):
        try:
            return next(int(p) for p in parts if p.isdigit())
        except StopIteration:
            return None

    def _handle_exit(self):
        self.got.play_audio_tts(
            "Shutting down. Goodbye!",
            voice_type=self.voice_type,
            wait=True
        )

    def _move_forward(self, amount):
        self.got.play_audio_tts(
            f"Moving forward {amount}",
            voice_type=self.voice_type,
            wait=True
        )
        self.got.mecanum_move_speed_times(0, 20, amount, 1)

    def _move_backward(self, amount):
        self.got.play_audio_tts(
            f"Moving backward {amount}",
            voice_type=self.voice_type,
            wait=True
        )
        self.got.mecanum_move_speed_times(1, 20, amount, 1)

    def _turn_left(self, amount):
        self.got.play_audio_tts(
            f"Turning left {amount} times",
            voice_type=self.voice_type,
            wait=True
        )
        self.got.mecanum_turn_speed_times(2, 30, amount, 2)

    def _turn_right(self, amount):
        self.got.play_audio_tts(
            f"Turning right {amount} times",
            voice_type=self.voice_type,
            wait=True
        )
        self.got.mecanum_turn_speed_times(3, 30, amount, 2)

    def _unknown_command(self):
        self.got.play_audio_tts(
            "Sorry, I didn't get that. Try commands like 'forward five' or 'left three'.",
            voice_type=self.voice_type,
            wait=True
        )


controller = VoiceController(got_interface=got,voice_type=0)
controller.run()


Raw command: 'Forward 10'
Parsed amount: 10
Raw command: 'Left 90'
Parsed amount: 90
Raw command: 'Exit'
